### 1. Data Preprocessing

In [1]:
import os
import numpy as np
import glob
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Set the path to the data directory
data_dir = r'C:\Users\VIVITHA NISANTHINI\Downloads\Data'

# Get the list of all .tif (case insensitive) files in the directory and its subdirectories
all_files = glob.glob(os.path.join(data_dir, '**', '*.tif'), recursive=True) + glob.glob(os.path.join(data_dir, '**', '*.TIF'), recursive=True)

# Separate the files into images and masks
image_paths = sorted([f for f in all_files if '_mask' not in f])
mask_paths = sorted([f for f in all_files if '_mask' in f])

# Print the number of images and masks found to ensure everything is working correctly
print(f"Found {len(image_paths)} images and {len(mask_paths)} masks")


# Split dataset into 80% training and 20% testing sets
train_images, test_images, train_masks, test_masks = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

# CLAHE preprocessing
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# Load and preprocess images
def load_data(image_paths, mask_paths):
    images = []
    masks = []
    for img_path, mask_path in zip(image_paths, mask_paths):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        img = apply_clahe(img)
        img = cv2.resize(img, (256, 256)) / 255.0
        mask = cv2.resize(mask, (256, 256)) / 255.0
        images.append(img)
        masks.append(mask)
    return np.array(images), np.array(masks)

train_images, train_masks = load_data(train_images, train_masks)
test_images, test_masks = load_data(test_images, test_masks)


Found 7858 images and 7858 masks


### 2. Model Implementation


### 3. Model Training and Evaluation

In [ ]:
# Expand dimensions to add channel for grayscale images
train_images = np.expand_dims(train_images, axis=-1)
train_masks = np.expand_dims(train_masks, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
test_masks = np.expand_dims(test_masks, axis=-1)

# Train the models
nested_unet.fit(train_images, train_masks, validation_data=(test_images, test_masks), epochs=10, batch_size=4)
attention_unet.fit(train_images, train_masks, validation_data=(test_images, test_masks), epochs=10, batch_size=4)

# Evaluate the models using DICE Score
def dice_score(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

nested_unet_dice = dice_score(test_masks, nested_unet.predict(test_images))
attention_unet_dice = dice_score(test_masks, attention_unet.predict(test_images))

print(f"Nested U-Net DICE Score: {nested_unet_dice}")
print(f"Attention U-Net DICE Score: {attention_unet_dice}")


### 4. Web Application Development
- FAST API Backend

In [ ]:
from fastapi import FastAPI, UploadFile, File
import uvicorn
import numpy as np
import cv2

app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Read image from file
    contents = await file.read()
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
    img = apply_clahe(img)
    img = cv2.resize(img, (256, 256)) / 255.0
    img = np.expand_dims(img, axis=(0, -1))

    # Make prediction
    prediction = attention_unet.predict(img)
    return {"prediction": prediction.tolist()}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


- Streamlit UI

In [ ]:
import streamlit as st
import requests
import numpy as np
from PIL import Image

st.title("Brain MRI Metastasis Segmentation")
uploaded_file = st.file_uploader("Choose a brain MRI image...", type="jpg")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded MRI Image.', use_column_width=True)
    
    # Make a request to the FastAPI server
    files = {'file': uploaded_file.getvalue()}
    response = requests.post("http://localhost:8000/predict/", files=files)

    if response.status_code == 200:
        prediction = np.array(response.json()["prediction"])
        st.image(prediction[0], caption='Metastasis Segmentation Result', use_column_width=True)
